# Dummy Provider Example and High Volume Robustness Testing

This notebook has two purposes: 

- Demonstrate the dummy feedback function provider which behaves like the
  huggingface provider except it does not actually perform any network calls and
  just produces constant results. It can be used to prototype feedback function
  wiring for your apps before invoking potentially slow (to run/to load)
  feedback functions.

- Test out high-volume record and feedback computation. To this end, we use the
  custom app which is dummy in a sense that it produces useless answers without
  making any API calls but otherwise behaves similarly to real apps, and the
  dummy feedback function provider.

In [ ]:
import asyncio
import concurrent
from pathlib import Path
import sys

from tqdm.auto import tqdm

# Add base dir to path to be able to access test folder.
base_dir = Path().cwd().parent.parent.resolve()
if str(base_dir) not in sys.path:
    print(f"Adding {base_dir} to sys.path")
    sys.path.append(str(base_dir))

In [ ]:
from trulens.core import TruSession

session = TruSession()
session.reset_database()

session.experimental_enable_feature("otel_tracing")

In [ ]:
from trulens.core import Feedback
from trulens.core import TruSession
from trulens.core.utils.threading import TP
from trulens.dashboard.run import run_dashboard
from trulens.feedback.dummy.provider import DummyProvider as DummyLLM
from trulens.providers.huggingface.provider import Dummy as DummyHugs

from examples.dev.dummy_app.app import DummyApp

tp = TP()

provider_hugs = DummyHugs(
    loading_prob=0.0,
    freeze_prob=0.0,  # we expect requests to have their own timeouts so freeze should never happen
    error_prob=0.0,
    overloaded_prob=0.0,
    rpm=10000,
    alloc=0,  # how much fake data to allocate during requests
    delay=0.1,
)

provider_llm = DummyLLM(
    loading_prob=0.0,
    freeze_prob=0.0,  # we expect requests to have their own timeouts so freeze should never happen
    error_prob=0.0,
    overloaded_prob=0.0,
    rpm=10000,
    alloc=0,  # how much fake data to allocate during requests
    delay=0.1,
)

session = TruSession()

session.reset_database()

run_dashboard(session, force=True, _dev=base_dir, port=8080)

In [ ]:
f_dummy1 = Feedback(provider_hugs.language_match).on_input_output()

f_dummy2 = Feedback(provider_hugs.positive_sentiment).on_output()

f_dummy3 = Feedback(provider_llm.sentiment).on_input()

# Synchronous Invocation

In [ ]:
# Create custom app:
ca = DummyApp(delay=0.0, alloc=0)

# Create trulens wrapper:
ta = session.App(
    ca,
    app_name="synchronous invoke",
    feedbacks=[f_dummy1, f_dummy2, f_dummy3],
)

with ta as recorder:
    res = ca.respond_to_query("hello there")
    print(res)

print(recorder.get().cost)

# Asynchronous Invocation

In [ ]:
# Create custom app:
ca = DummyApp(delay=0.0, alloc=0)

# Create trulens wrapper:
ta = session.App(
    ca,
    app_name="asynchronous invoke",
    feedbacks=[f_dummy1, f_dummy2, f_dummy3],
)

async with ta as recorder:
    res = await ca.arespond_to_query("hello there")

    print(res)

print(recorder.get().cost)

# Sequential App Invocation

In [ ]:
ca = DummyApp(delay=0.0, alloc=0, use_parallel=True)

ta = session.App(
    ca,
    app_name="sequential invoke",
    feedbacks=[f_dummy1, f_dummy2, f_dummy3],
)

for i in tqdm(range(10), desc="invoking app"):
    with ta as recorder:
        ca.respond_to_query(f"hello {i}")

    rec = recorder.get()
    assert rec is not None

    print(rec.record_id)

for res in tqdm(
    ta.wait_for_feedback_results(), desc="waiting for feedback results"
):
    print(res)

# Parallel Feedback Evaluation

In [ ]:
futures = []
num_tests = 100
good = 0
bad = 0


def test_feedback(msg):
    return msg, provider_hugs.positive_sentiment(msg)


for i in tqdm(range(num_tests), desc="starting feedback task"):
    futures.append(tp.submit(test_feedback, msg="good"))

prog = tqdm(concurrent.futures.as_completed(futures), total=num_tests)

for f in prog:
    try:
        res = f.result()
        good += 1

        assert res[0] == "good"

        prog.set_description_str(f"{good} / {bad}")
    except Exception:
        bad += 1
        prog.set_description_str(f"{good} / {bad}")

# Parallel Feedback Evaluation with Deferred Mode

In [ ]:
session.start_evaluator(restart=True)

In [ ]:
ca = DummyApp(delay=0.0, alloc=0, use_parallel=True)

ta = session.App(
    ca,
    app_name="sequential invoke with deferred feedback",
    feedbacks=[f_dummy1, f_dummy2, f_dummy3],
    feedback_mode="deferred",
)

for i in tqdm(range(10), desc="invoking app"):
    with ta as recorder:
        ca.respond_to_query(f"hello {i}")

    rec = recorder.get()
    assert rec is not None

# Parallel App Invocation using Threads

In [ ]:
from threading import Thread  # must be imported after trulens

# Create custom app:
ca = DummyApp(delay=0.1, alloc=0, use_parallel=True)

# Create trulens wrapper:
ta = session.App(
    ca,
    app_name="threaded parallel invoke",
    feedbacks=[f_dummy1, f_dummy2, f_dummy3],
)


def run_query(q):
    with ta as recorder:
        ca.respond_to_query(q)

    rec = recorder.get()
    assert rec is not None

    results = list(rec.feedback_results)

    ret = f"run_query {q}, has {len(results)} feedbacks"
    print(ret)

    return ret


threads = []

for i in tqdm(range(10), desc="starting app task"):
    t = Thread(target=run_query, args=(f"hello {i}",))
    t.start()
    threads.append(t)

for t in tqdm(threads, desc="waiting for recording threads to finish"):
    t.join()

for record in tqdm(
    ta.wait_for_feedback_results(), desc="waiting for feedbacks to finish"
):
    print(
        f"record {record.record_id} has {len(record.feedback_results)} feedbacks"
    )

# Parallel App Invocation using Tasks

In [ ]:
# Create custom app:
ca = DummyApp(
    delay=0.1,
    alloc=0,
    use_parallel=True,  # need to enable this for DummyApp to use tasks internally
)

# Create trulens wrapper:
ta = session.App(
    ca,
    # app_name="customapp",
    app_name="async parallel invoke",
    feedbacks=[f_dummy1, f_dummy2, f_dummy3],
)


async def arun_query(q):
    print(f"starting {q}")
    async with ta as recorder:
        print("awaiting respond")
        await ca.arespond_to_query(q)

        print("got respond")

    print("out of context")

    rec = recorder.get()
    assert rec is not None

    ret = f"run_query {q}, has {len(rec.feedback_results)} feedbacks"
    print(ret)

    return ret


loop = asyncio.get_event_loop()
tasks = []

for i in tqdm(range(10), desc="starting app task"):
    t = loop.create_task(arun_query(f"hello {i}"))
    tasks.append(t)

async for t in tqdm(
    asyncio.as_completed(tasks), desc="awaiting task", total=len(tasks)
):  # have to use sync loop if python < 3.13
    await t

for record in tqdm(
    ta.wait_for_feedback_results(), desc="waiting for feedbacks to finish"
):
    print(
        f"record {record.record_id} has {len(record.feedback_results)} feedbacks"
    )